<a href="https://colab.research.google.com/github/harsh10979/project-1/blob/main/Project_2_on_exact_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.8 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Create a Bayesian Network for car diagnostics
model = BayesianNetwork([
    ('Battery', 'StartingSystem'),
    ('FuelSystem', 'StartingSystem'),
    ('StartingSystem', 'CarStarts')
])

# Define Conditional Probability Distributions (CPDs)
cpd_battery = TabularCPD(variable='Battery', variable_card=2, values=[[0.9], [0.1]])

cpd_fuel_system = TabularCPD(variable='FuelSystem', variable_card=2, values=[[0.95], [0.05]])

cpd_starting_system = TabularCPD(
    variable='StartingSystem',
    variable_card=2,
    values=[
        [0.99, 0.01, 0.01, 0.0],
        [0.01, 0.99, 0.99, 1.0]
    ],
    evidence=['Battery', 'FuelSystem'],
    evidence_card=[2, 2]
)

cpd_car_starts = TabularCPD(
    variable='CarStarts',
    variable_card=2,
    values=[
        [0.99, 0.01],
        [0.01, 0.99]
    ],
    evidence=['StartingSystem'],
    evidence_card=[2]
)

# Add CPDs to the model
model.add_cpds(cpd_battery, cpd_fuel_system, cpd_starting_system, cpd_car_starts)

# Check if the model is valid
assert model.check_model()

# Perform exact inference
inference = VariableElimination(model)

# Updated diagnose_car function
def diagnose_car(car_starts, battery_ok=None, fuel_system_ok=None):
    evidence = {'CarStarts': car_starts}
    query_variables = ['Battery', 'FuelSystem', 'StartingSystem']

    if battery_ok is not None:
        evidence['Battery'] = battery_ok
        query_variables.remove('Battery')
    if fuel_system_ok is not None:
        evidence['FuelSystem'] = fuel_system_ok
        query_variables.remove('FuelSystem')

    results = {}
    for var in query_variables:
        prob = inference.query(variables=[var], evidence=evidence)
        results[f'{var} OK'] = prob.values[1]

    # Add known values to results
    if battery_ok is not None:
        results['Battery OK'] = battery_ok
    if fuel_system_ok is not None:
        results['Fuel System OK'] = fuel_system_ok

    return results

# Test the model
print("Probability of components being OK when:")
print("\nCar doesn't start:")
result = diagnose_car(car_starts=0)
for component, prob in result.items():
    print(f"{component}: {prob:.2f}")

print("\nCar doesn't start and battery is confirmed bad:")
result = diagnose_car(car_starts=0, battery_ok=0)
for component, prob in result.items():
    print(f"{component}: {prob:.2f}")

print("\nCar starts normally:")
result = diagnose_car(car_starts=1)
for component, prob in result.items():
    print(f"{component}: {prob:.2f}")


Probability of components being OK when:

Car doesn't start:
Battery OK: 0.00
FuelSystem OK: 0.00
StartingSystem OK: 0.00

Car doesn't start and battery is confirmed bad:
FuelSystem OK: 0.00
StartingSystem OK: 0.00
Battery OK: 0.00

Car starts normally:
Battery OK: 0.62
FuelSystem OK: 0.31
StartingSystem OK: 0.95
